In [2]:
from langchain.memory.chat_message_histories import ZepChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool
from uuid import uuid4
import pprint

In [3]:
# Set this to your Zep server URL
ZEP_API_URL = "http://localhost:8000"

session_id = str(uuid4())  # This is a unique identifier for the user

In [4]:
# Load your OpenAI key from a .env file
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
search = WikipediaAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to search online for answers. You should ask targeted questions"
    ),
]

# Set up Zep Chat History
zep_chat_history = ZepChatMessageHistory(
    session_id=session_id,
    url=ZEP_API_URL,
)

# Use a standard ConversationBufferMemory to encapsulate the Zep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=zep_chat_history
)

# Initialize the agent
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [6]:
# Preload some messages into the memory. The default message window is 12 messages. We want to push beyond this to demonstrate auto-summarization.
test_history = [
    {"role": "human", "content": "Who was Octavia Butler?"},
    {
        "role": "ai",
        "content": (
            "Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American"
            " science fiction author."
        ),
    },
    {"role": "human", "content": "Which books of hers were made into movies?"},
    {
        "role": "ai",
        "content": (
            "The most well-known adaptation of Octavia Butler's work is the FX series"
            " Kindred, based on her novel of the same name."
        ),
    },
    {"role": "human", "content": "Who were her contemporaries?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler's contemporaries included Ursula K. Le Guin, Samuel R."
            " Delany, and Joanna Russ."
        ),
    },
    {"role": "human", "content": "What awards did she win?"},
    {
        "role": "ai",
        "content": (
            "Octavia Butler won the Hugo Award, the Nebula Award, and the MacArthur"
            " Fellowship."
        ),
    },
    {
        "role": "human",
        "content": "Which other women sci-fi writers might I want to read?",
    },
    {
        "role": "ai",
        "content": "You might want to read Ursula K. Le Guin or Joanna Russ.",
    },
    {
        "role": "human",
        "content": (
            "Write a short synopsis of Butler's book, Parable of the Sower. What is it"
            " about?"
        ),
    },
    {
        "role": "ai",
        "content": (
            "Parable of the Sower is a science fiction novel by Octavia Butler,"
            " published in 1993. It follows the story of Lauren Olamina, a young woman"
            " living in a dystopian future where society has collapsed due to"
            " environmental disasters, poverty, and violence."
        ),
    },
]

for msg in test_history:
    zep_chat_history.append(
        HumanMessage(content=msg["content"])
        if msg["role"] == "human"
        else AIMessage(content=msg["content"])
    )

# Inspect the Zep memory
**Note the summary, and that the history has been enriched with entities, token counts, UUIDs, and timestamps.**

Summaries are biased towards the most recent messages.

In [7]:
def print_messages(messages):
    for m in messages:
        print(m.dict())


print(zep_chat_history.zep_summary)
print("\n")
print_messages(zep_chat_history.zep_messages)

None


{'role': 'human', 'content': 'Who was Octavia Butler?', 'uuid': '8da13e94-ff3b-4478-a8b4-5eef337e7505', 'created_at': '2023-05-26T17:48:07.773489Z', 'token_count': 0, 'metadata': None}
{'role': 'ai', 'content': 'Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American science fiction author.', 'uuid': 'e08d507c-3c91-4996-af8c-89fb6f6b5703', 'created_at': '2023-05-26T17:48:07.835229Z', 'token_count': 0, 'metadata': None}
{'role': 'human', 'content': 'Which books of hers were made into movies?', 'uuid': '34dcf07c-c6a8-43fc-81e5-e4ca2e077586', 'created_at': '2023-05-26T17:48:07.847338Z', 'token_count': 0, 'metadata': None}
{'role': 'ai', 'content': "The most well-known adaptation of Octavia Butler's work is the FX series Kindred, based on her novel of the same name.", 'uuid': 'fe733c07-75cf-481d-a7a6-33e0985ed4f0', 'created_at': '2023-05-26T17:48:07.861084Z', 'token_count': 0, 'metadata': None}
{'role': 'human', 'content': 'Who were her contemporaries?', 'uuid': '

In [8]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(zep_chat_history.zep_messages[1].dict())

{   'content': 'Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was '
               'an American science fiction author.',
    'created_at': '2023-05-26T17:48:07.835229Z',
    'metadata': {   'system': {   'entities': [   {   'Label': 'PERSON',
                                                      'Matches': [   {   'End': 22,
                                                                         'Start': 0,
                                                                         'Text': 'Octavia '
                                                                                 'Estelle '
                                                                                 'Butler'}],
                                                      'Name': 'Octavia Estelle '
                                                              'Butler'},
                                                  {   'Label': 'DATE',
                                                      'Matches': [   {  

In [9]:
agent_chain.run(
    input="Recommend science fiction books I should read that are similar to Octavia Butler's Parable of the Sower. Research online."
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: Science fiction books similar to Octavia Butler's Parable of the Sower
Observation: Page: Octavia E. Butler
Summary: Octavia Estelle Butler (June 22, 1947 – February 24, 2006) was an American science fiction author and a multiple recipient of the Hugo and Nebula awards. In 1995, Butler became the first science-fiction writer to receive a MacArthur Fellowship.Born in Pasadena, California, Butler was raised by her widowed mother. Extremely shy as a child, Butler found an outlet at the library reading fantasy, and in writing. She began writing science fiction as a teenager. She attended community college during the Black Power movement. While  participating in a local writer's workshop, she was encouraged to attend the Clarion Workshop, then held in Pennsylvania, which focused on science fiction.She soon sold her first stories and by the late 1970s had become sufficiently successful 

"Based on Octavia Butler's Parable of the Sower, some other science fiction books you might want to read include Ursula K. Le Guin's The Left Hand of Darkness, Joanna Russ's The Female Man, and Samuel R. Delany's Dhalgren."

In [11]:
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(zep_chat_history.zep_messages[-1].dict())

{   'content': "Based on Octavia Butler's Parable of the Sower, some other "
               'science fiction books you might want to read include Ursula K. '
               "Le Guin's The Left Hand of Darkness, Joanna Russ's The Female "
               "Man, and Samuel R. Delany's Dhalgren.",
    'created_at': '2023-05-26T17:48:22.242866Z',
    'metadata': {   'system': {   'entities': [   {   'Label': 'PERSON',
                                                      'Matches': [   {   'End': 25,
                                                                         'Start': 9,
                                                                         'Text': 'Octavia '
                                                                                 "Butler's"}],
                                                      'Name': 'Octavia '
                                                              "Butler's"},
                                                  {   'Label': 'WORK_OF_ART',
  